В это тетрадке я пытаюсь выжать приличный скор из фейсбуковского fastText'a.  Для этого я опредляю несколько оберток, чтобы получить более привичный интерфейс

In [1]:
import fastText
import pandas as pd
import numpy as np
import nltk
from pymystem3 import Mystem
from pathlib import Path
from sklearn.metrics import accuracy_score

In [12]:
# fastext не умеет предсказывать несколько примеров сразу
# поэтому вызваем его в цикле, заодно обрезая лишнее
def predict_ft(model, texts):
    labels = np.array([model.predict(text.strip('\n'))[0][0].strip('__label__') for text in texts])
    return labels.astype(int)

In [13]:
# простая функция для кроссвалидация, использующая заранее созраненные в файлы фолды в подходящем формате
def cross_val_score(data_path,
                    scorer, model_params):
    
    data_dir = Path(data_path)
    n_files = len(list(data_dir.glob('*.txt*')))
    # на каждый фолд должно приходится три файла: train, test-text, test-label
    assert n_files % 3 == 0
    n_splits = n_files // 3
    
    scores = []
    
    for fold in range(n_splits):
        train_path = Path(data_path, f'train_{fold}.txt')
        test_path = Path(data_path, f'test_{fold}.txt')
        label_path = Path(test_path.parent, test_path.name + '.label')
        with open(test_path) as f:
            test_texts = f.readlines()
            
        test_labels = pd.read_csv(label_path, header=None,names=['label'])
        
        model = fastText.train_supervised(train_path.as_posix(), **model_params)
        
        preds = predict_ft(model, test_texts)
        scores.append(scorer(test_labels, preds))
    return np.mean(scores)

In [14]:
# велосипед для перебора параметров

def random_search(data_path,
                  param_grid, scorer,  n_trials=10):
    best_score  = 0 
    best_paramss = {}
    scores = {}
    while n_trials:
        model_params = {
            k: np.random.choice(param_grid[k])
            for k in param_grid
        }
        
        # зарардкоженная проверка на наличие готовых ембеддингов
        if model_params['pretrainedVectors'] != '':
            model_params['dim'] = 300
            
        # гарантируем чтобы maxn был не меньше minn
        model_params['maxn'] = max(model_params['maxn'], model_params['minn'])
        
        # проверяем что мы еще не пробовали этот набор параметров
        if tuple(model_params.items()) not in scores:
            
            score = cross_val_score(data_path,
                                   scorer, model_params)
            scores[tuple(model_params.items())] = score
            n_trials -= 1
            if score > best_score:
                best_score = score
                best_params = model_params

    return best_score, best_params, scores

In [ ]:
param_grid = {
    'minCount': range(1,6) ,
    'wordNgrams': range(1,4),
    'minn': range(3) ,
    'maxn': range(5),
    'epoch': [15,20],
    'thread': [3],
    'dim': [100, 150, 200, 250],
    'lr': np.linspace(0.1, 0.2, 50),
    'lrUpdateRate': np.arange(1,10)*100,
    'pretrainedVectors': ['', '../embeddings/ft_native_300_ru_wiki_lenta_lemmatize.vec']
}
best_score_2, best_params_2, scores_2 = random_search('../data/ft/lemmatized/kfolds',
                                                      param_grid, accuracy_score, n_trials=10)

In [ ]:
param_grid = {
    'minCount': range(1,6) ,
    'wordNgrams': range(1,4),
    'minn': range(3) ,
    'maxn': range(5),
    'epoch': [15,20],
    'thread': [3],
    'dim': [100, 150, 200, 250],
    'lr': np.linspace(0.1, 0.2, 50),
    'lrUpdateRate': np.arange(1,10)*100,
    'pretrainedVectors': ['']
}
best_score_3, best_params_3, scores_3 = random_search('../data/ft/lemmatized/kfolds',
                                                      param_grid, accuracy_score, n_trials=10)

In [19]:
best_params_3

{'minCount': 3,
 'wordNgrams': 3,
 'minn': 0,
 'maxn': 4,
 'epoch': 20,
 'thread': 3,
 'dim': 100,
 'lr': 0.1979591836734694,
 'lrUpdateRate': 300,
 'pretrainedVectors': ''}

In [21]:
best_score_3

0.871888863726744

In [22]:
best_score_2

0.8728234039414533

In [ ]:
param_grid = {
    'minCount': range(1,6) ,
    'wordNgrams': range(1,4),
    'minn': [2] ,
    'maxn': [5],
    'epoch': [5, 10, 15],
    'thread': [6],
    'dim': [50, 100, 200],
    'lr': np.linspace(0.05, 0.2, 200),
    'lrUpdateRate': np.arange(1,10)*100,
}

In [12]:
best_score, best_params, scores = random_search('../data/ft/lemmatized/kfolds', '../data/json/kfolds/', 'lemmatized',
              param_grid, accuracy_score)

In [13]:
best_score

0.8728872657040551

In [14]:
best_params

{'minCount': 1,
 'wordNgrams': 2,
 'minn': 2,
 'maxn': 5,
 'epoch': 15,
 'thread': 6,
 'dim': 200,
 'lr': 0.17512562814070354,
 'lrUpdateRate': 400}

Не очень впечатляющие результаты, возможн стоило сначала обучить свои ембеддинги на трейне, и далее использовать их, а также перебирать параметры подольше. Но другие модели выдают лучший результат при меньших затратах времени, так что я решил оставить fastText.